<a href="https://colab.research.google.com/github/seraffin/FailOmen/blob/master/Mariusz/workingNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

# Helper libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from google.colab import drive


checkGPUpresence()

# Utilities

In [0]:
# Evaluation function

def evaluation(additionalPredictions):
  
  lenght = len(yTest)

  failPositions = [[] for y in range(lenght)]

  for i, a in enumerate(yTest):

    for j, b in enumerate(a):
      if b == 0 : failPositions[i].append(j);

  predictionsTemp = predictions.copy()
  predictionPositions = [[] for y in range(lenght)]


  for i, a in enumerate(predictionsTemp):

    if len(failPositions[i]) != 0:
      for j in range(len(failPositions[i]) + additionalPredictions):
        argmin = np.argmin(a)
        predictionPositions[i].append(argmin)
        predictionsTemp[i][argmin] = 1

  predictionHits = [[] for y in range(lenght)]

  for i, a in enumerate(failPositions):
    count = 0
    for j, b in enumerate(a):


      for c in predictionPositions[i]:
  #      predictionHits[i].append(predictions[i][c].copy())
  #      print(predictions[i][c])
  #      print(predictions[i][c])
  #      print (b, c)
        if c == b : count += 1

    if len(failPositions) != 0:
      predictionHits[i].insert(0,count)


  failsCount = 0
  hitsCount = 0
  for i, a in enumerate(yTest):
    j = 0

    for k, b in enumerate(a):
      if b < 1.0 : j += 1

  #  if j > 0 :
  #    print (i, '.', j, predictionHits[i])


    failsCount += j
    hitsCount += predictionHits[i][0]

  print('Percentage of fails predicted', hitsCount / failsCount * 100, '%')

In [0]:
#Distribution of argmins through all the predictions
  
def argminsDistribution():
  
    i = 1
    tab = [0] * 542
    for a in predictions:
      j = 0
      for b in a:
        if b < 1.0 : j = j + 1

      #print (i, '. ', j, np.argmin(a))
      tab[np.argmin(a)] += 1
      i = i + 1

    i = 0
    for a in tab:
      if a > 0 : print ('position', i, '\targmin count', a)
      i += 1

In [0]:
def printFailDistributionStats(y):
  
  failCount = 0
  passCount = 0
  failBuildCount = 0

  for a in y:
    if a[np.argmin(a)] == 0 : failBuildCount += 1
    for b in a:
      if b == 0 : failCount += 1
      if b == 1 : passCount += 1
  print ('failBuildCount', failBuildCount)
  print ('passBuildCount', len(y) - failBuildCount)
  print ('failBuild share ', failBuildCount / len(y) * 100, '%', sep='')
  print ('failCount', failCount)
  print ('passCount', passCount)
  print ('fail share ', failCount / (failCount + passCount) * 100, '%', sep='')

In [0]:
# Create a list of single member classes positions

def listSingleMemberClassesPositions(refY, printFlag):

  yList = refY.tolist()
  yDistinct = [ele for ind, ele in enumerate(yList) if ele not in yList[:ind]]

  classesPopulation = []
  singleMemberClassesPositions = []
  noSingleMemberClasses = 0

  for i, a in enumerate(yDistinct):
    population = 0
    memberPosition = 0
    for j, b in enumerate(yList):
      if a == b: 
        population += 1
        memberPosition = j

    classesPopulation.append(population)
  #  print(i, population)
    if population == 1: 
      noSingleMemberClasses += 1
      singleMemberClassesPositions.append(memberPosition)

  if printFlag == True: 
    print(noSingleMemberClasses)
  else:
    return singleMemberClassesPositions

In [0]:
# Double the single member classes for stratified sampling

def appendToDataset(refSingleMemberClassesPositions, refDataset):
  
  newDataset = pd.DataFrame(columns=list(refDataset))
  for i, a in enumerate(refSingleMemberClassesPositions):
    newDataset = newDataset.append(refDataset.iloc[a, :])
  
  return refDataset.append(newDataset, ignore_index=True)

In [0]:
def checkGPUpresence():

  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
  print('Found GPU at: {}'.format(device_name))

# Prepare data

In [0]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#Importing dataset

#dataset = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataToML_newConcept.csv', index_col=False)
dataset = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataToML_full2.csv', index_col=False)

In [0]:
# Split data set into parameters and labels

x = dataset.iloc[:, 0:23739].values
y = dataset.iloc[:, 23739:24281].values

In [0]:
# Random sampling

xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2)

listSingleMemberClassesPositions(yTrain, True)
listSingleMemberClassesPositions(yTest, True)


In [0]:
# Stratified sampling

dataset = appendToDataset(listSingleMemberClassesPositions(y, False), dataset)

x = dataset.iloc[:, 0:23739].values
y = dataset.iloc[:, 23739:24281].values

from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=None)

for train_index, test_index in sss.split(x, y):
  xTrain, xTest = x[train_index], x[test_index]
  yTrain, yTest = y[train_index], y[test_index]
  
listSingleMemberClassesPositions(yTrain, True)
listSingleMemberClassesPositions(yTest, True)

80
130


# Prepare and train model

In [0]:
model = Sequential()
# Adding the input layer and the first hidden layer
model.add(Dense(6070, activation="sigmoid", input_dim=23739, kernel_initializer="uniform"))
# Adding the second hidden layer
#model.add(Dense(12140, activation = "sigmoid", kernel_initializer="uniform"))
# Adding the output layer
model.add(Dense(542, activation="sigmoid", kernel_initializer="uniform"))

In [0]:
model.compile(optimizer = tf.train.MomentumOptimizer(learning_rate = 0.01, momentum = 0.4), loss = 'binary_crossentropy', metrics = ['accuracy'])

model.fit(xTrain, yTrain, batch_size = 20, epochs = 15)

Epoch 1/15
4005/4005 [==============================] - 13s 3ms/step - loss: 0.3054 - acc: 0.9157
Epoch 2/15
4005/4005 [==============================] - 11s 3ms/step - loss: 0.0803 - acc: 0.9994
Epoch 3/15
4005/4005 [==============================] - 11s 3ms/step - loss: 0.0476 - acc: 0.9994
Epoch 4/15
4005/4005 [==============================] - 11s 3ms/step - loss: 0.0343 - acc: 0.9994
Epoch 5/15
4005/4005 [==============================] - 11s 3ms/step - loss: 0.0272 - acc: 0.9994
Epoch 6/15
4005/4005 [==============================] - 11s 3ms/step - loss: 0.0227 - acc: 0.9994
Epoch 7/15
4005/4005 [==============================] - 11s 3ms/step - loss: 0.0197 - acc: 0.9994
Epoch 8/15
4005/4005 [==============================] - 11s 3ms/step - loss: 0.0174 - acc: 0.9994
Epoch 9/15
4005/4005 [==============================] - 11s 3ms/step - loss: 0.0158 - acc: 0.9994
Epoch 10/15
4005/4005 [==============================] - 11s 3ms/step - loss: 0.0144 - acc: 0.9994
Epoch 11/15
4005/40

# Evaluate

In [0]:
predictions = model.predict(xTest)

In [0]:
#Distributions of argmins through all the predictions
  
def argminsDistribution(): 
    i = 1
    tab = [0] * 542
    for a in predictions:
      j = 0
      for b in a:
        if b < 1.0 : j = j + 1

      #print (i, '. ', j, np.argmin(a))
      tab[np.argmin(a)] += 1
      i = i + 1

    i = 0
    for a in tab:
      if a > 0 : print ('position', i, '\targmin count', a)
      i += 1

In [0]:
#failsCount()

In [0]:
argminsDistribution()

position 71 	argmin count 11
position 200 	argmin count 38
position 462 	argmin count 953


In [0]:
for i in range(10):
  print(i, ' ', end='')
  evaluation(i)

0  Percentage of fails predicted 5.2805280528052805 %
1  Percentage of fails predicted 7.2607260726072615 %
2  Percentage of fails predicted 8.91089108910891 %
3  Percentage of fails predicted 10.891089108910892 %
4  Percentage of fails predicted 12.871287128712872 %
5  Percentage of fails predicted 13.531353135313532 %
6  Percentage of fails predicted 15.51155115511551 %
7  Percentage of fails predicted 17.16171617161716 %
8  Percentage of fails predicted 17.491749174917494 %
9  Percentage of fails predicted 20.462046204620464 %
